<a href="https://colab.research.google.com/github/LexMed-AI/st_deepgram_playground/blob/main/Nick's_GPT_4_DeepGram_Speaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install requests ffmpeg-python
! pip install deepgram-sdk==2.12.0
! pip install asyncio
! pip install time
! pip install openai
!pip install nest_asyncio

ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time


In [ ]:
from deepgram import Deepgram
import asyncio, json, os
from google.colab import userdata

dg_key = userdata.get('dg_key')
dg = Deepgram(dg_key)

'''
The most common audio formats and encodings we support
include mp3, mp4, mp2, aac, wav, flac, pcm, m4a, ogg, opus, and webm,
So feel free to adjust the `MIMETYPE` variable as needed
'''
MIMETYPE = 'ogg'

#Note: You can use '.' if your audio is in the root
DIRECTORY = '.'


# Feel free to modify your model's parameters as you wish!
options = {
    "smart_format": True,
    "diarize": True,
    "model": 'enhanced',
}

#This function is what calls on the model to transcribe
def main():
    audio_folder = os.listdir(DIRECTORY)
    for audio_file in audio_folder:
        if audio_file.endswith(MIMETYPE):
          with open(f"{DIRECTORY}/{audio_file}", "rb") as f:
              source = {"buffer": f, "mimetype":'audio/'+MIMETYPE}
              res = dg.transcription.sync_prerecorded(source, options)
              with open(f"./{audio_file[:-4]}.json", "w") as transcript:
                  json.dump(res, transcript, indent=4)
    return

main()

In [ ]:
import json
import os

DIRECTORY = '/content/'  # Define the directory to scan for JSON files

def format_transcript_data(raw_transcript):
    transcript_data = []
    items = raw_transcript.get('results', {}).get('channels', [{}])[0].get('alternatives', [{}])[0]
    paragraphs = items.get('paragraphs', {}).get('paragraphs', [])

    if not paragraphs:
        print("No paragraphs found in transcript.")
        return []

    try:
        last_speaker_id = None
        id_counter = 0

        for paragraph in paragraphs:
            speaker_id = str(paragraph['speaker'])
            transcript_text = ' '.join(sentence['text'] for sentence in paragraph.get('sentences', []))
            start_time = paragraph.get('sentences', [{}])[0].get('start', 0)

            # Determine if the current speaker is the same as the last one and spoken consecutively
            if speaker_id != last_speaker_id:
                id_counter += 1  # Increment ID since a new speaker or non-consecutive speech has started
                last_speaker_id = speaker_id

            # Check if an entry for the current ID already exists
            if id_counter not in [entry['id'] for entry in transcript_data]:
                # Create a new entry for this ID
                transcript_object = {
                    'id': id_counter,
                    'ts': start_time,
                    'speaker': "",
                    'text': transcript_text
                }
                transcript_data.append(transcript_object)
            else:
                # Append text to the existing entry for this ID
                for entry in transcript_data:
                    if entry['id'] == id_counter:
                        entry['text'] += " " + transcript_text
                        break

        return transcript_data
    except Exception as error:
        print(f"Error processing the transcript data: {error}")
        return []

def create_transcript(input_filename):
    with open(input_filename, "r") as file:
        raw_transcript = json.load(file)
        formatted_data = format_transcript_data(raw_transcript)
        output_filename = "Parsed_JSON.json"  # Set the desired output file name
        with open(output_filename, "w") as outfile:
            json.dump(formatted_data, outfile, indent=4)

def print_transcript():
    for filename in os.listdir(DIRECTORY):
        if filename.endswith('.json'):
            create_transcript(filename)
            print(f"Transcript written to Parsed_JSON.json")

if __name__ == '__main__':
    print_transcript()


Transcript written to Parsed_JSON.json


In [ ]:
import json
import math
import os
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def count_tokens(text):
    return len(text.split())

def chunk_transcript(parsed_json_file, output_directory, max_tokens=700, overlap_tokens=200):
    try:
        with open(parsed_json_file, 'r') as file:
            transcript_data = json.load(file)

        if not transcript_data:
            logger.warning("No data found in the file.")
            return

        logger.info(f"Total entries in data: {len(transcript_data)}")

        chunks = []
        current_chunk = []
        current_token_count = 0
        overlap_entries = []

        for entry in transcript_data:
            entry_text = entry['text']
            entry_token_count = count_tokens(entry_text)

            if current_token_count + entry_token_count <= max_tokens:
                current_chunk.append(entry)
                current_token_count += entry_token_count
            else:
                chunks.append(current_chunk)
                overlap_token_count = sum(count_tokens(e['text']) for e in overlap_entries)
                current_chunk = overlap_entries + [entry]
                current_token_count = overlap_token_count + entry_token_count
                overlap_entries = []

            if current_token_count >= max_tokens - overlap_tokens:
                overlap_entries = current_chunk[-2:]

        if current_chunk:
            chunks.append(current_chunk)

        num_chunks = len(chunks)
        logger.info(f"Total chunks created: {num_chunks}")

        num_digits = int(math.log10(num_chunks)) + 1 if num_chunks > 0 else 1

        for i, chunk in enumerate(chunks, start=1):
            chunk_filename = f"Parsed_JSON_Chunk_{i:0{num_digits}d}.json"
            chunk_filepath = os.path.join(output_directory, chunk_filename)
            with open(chunk_filepath, 'w') as file:
                json.dump(chunk, file, indent=4)
            logger.info(f"Chunk saved to {chunk_filepath}")

    except Exception as e:
        logger.error(f"Error occurred during chunking: {str(e)}")

# Specify the output directory where the chunked files should be saved
output_directory = '/content/chunked_files/'

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Usage
parsed_json_file = 'Parsed_JSON.json'
chunk_transcript(parsed_json_file, output_directory)

In [ ]:
system_prompt = json.dumps({
    "role": "Improve speaker diarization and formatting for ASR-generated Social Security Disability hearing transcripts.",
    "input": {
        "format": "JSON array",
        "fields": ["id", "ts", "speaker", "text"]
    },
    "tasks": [
        "Rectify spelling, titles, and terminology in the context of a Social Securirty Disability hearing",
        "Determine speakers labels using contextual information from dialog"
    ],
    "format": {
        "timestamps": "Format: [hh:mm:ss]",
        "correct_speaker_labels": {
            "Administrative Law Judge": "ALJ",
            "Claimant": "Clmt",
            "Attorney": "Atty",
            "Vocational Expert": "VE",
            "Medical Expert": "ME",
            "Hearing Reporter": "HR",
            "Witnesses": ["W1", "W2", "W3"],
        }
    },
    "corrections": {
        "spelling_punctuation": "Rectify mistakes, assure proper capitalization and punctuation.",
        "context": "Use context to rectify inaccuracies, align with hearing subjects.",
        "jargon": "Rectify inaccurate legal, medical, and occupational terms based on the context of SSA disability hearing."
    },
    "standardize": {
        "ssn": "Format: ###-##-####",
        "dot": "Format: ###.###-###",
        "dates": "Format: mm/dd/yyyy",
        "timestamp": "Format: hh:mm:ss",
        "titles": "Standardize: Ms., Mr., Dr., etc."
    },
    "guidelines": {
        "anonymity": "Keep LLM status anonymous and avoid disclaimers regarding ethics or confidentiality.",
        "integrity": "Stick to federal court standards.",
        "accuracy": "No placeholders or additional dialogue.",
        "output": "Return ONLY a JSON array of objects with 'id', 'ts', 'speaker', and 'text' fields, like this: [{\"id\": 0, \"ts\": \"...\", \"speaker\": \"...\", \"text\": \"...\"}, ...]"
    }
})

In [ ]:
import os
import json
import logging
import openai
import asyncio
import aiohttp
import time
from google.colab import userdata
from tqdm import tqdm
import nest_asyncio

nest_asyncio.apply()


# Initialize logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Retrieve the API key from Colab's secret manager
api_key = userdata.get('openai_api_key')
if not api_key:
    logging.error("API key not found. Please ensure it's set in Colab's secret manager.")
    raise ValueError("API key not found")

# Initialize OpenAI GPT-4 client
openai.api_key = api_key

async def send_to_gpt4(system_prompt, chunk_data, model="gpt-4-turbo-2024-04-09"):
    """Send structured system description to GPT-4 and return the response."""
    try:
        response = await asyncio.to_thread(openai.chat.completions.create,
            model=model,
            messages=[
                {
                    "role": "system",
                    "content": "You are an excellent optomizer of raw ARS transcripts.  Your output is limited to JSON."
                },
                {
                    "role": "user",
                    "content": system_prompt
                },
                {
                    "role": "user",
                    "content": json.dumps(chunk_data)
                }
            ]
        )
        if response.choices:
            return response.choices[0].message.content
        else:
            logging.info("No content returned in the response.")
            return None
    except Exception as e:
        logging.error(f"Error sending data to GPT-4: {e}")
        return None

DIRECTORY = '/content/'

async def fetch_response(session, chunk_file):
    try:
        with open(chunk_file, 'r') as file:
            chunk_data = json.load(file)
        response = await send_to_gpt4(system_prompt, chunk_data)
        return response
    except Exception as e:
        logging.error(f"Error processing chunk file {chunk_file}: {e}")
        return None

async def process_chunks_and_aggregate(chunked_files_directory, output_file="finalized_transcript.json"):
    chunked_files = sorted([os.path.join(chunked_files_directory, file) for file in os.listdir(chunked_files_directory) if file.endswith('.json')])
    responses = []

    async with aiohttp.ClientSession() as session:
        tasks = []
        for chunk_file in tqdm(chunked_files, desc="Processing chunks"):
            task = asyncio.ensure_future(fetch_response(session, chunk_file))
            tasks.append(task)
            responses = await asyncio.gather(*tasks, return_exceptions=True)

    aggregated_json = []
    for response in tqdm(responses, desc="Aggregating responses"):
        if response:
            try:
                chunk_data = json.loads(response)
                aggregated_json.extend(chunk_data)
            except Exception as e:
                logging.error(f"Error parsing response: {e}")

    # Sort the aggregated JSON based on the 'id' field
    aggregated_json = sorted(aggregated_json, key=lambda x: x['id'])

    # Write the aggregated JSON to the output file
    with open(output_file, 'w') as file:
        json.dump(aggregated_json, file, indent=2)

    return aggregated_json

if __name__ == "__main__":
    start_time = time.time()
    chunked_files_directory = '/content/chunked_files'  # Specify the directory containing the chunked files
    aggregated_json = asyncio.get_event_loop().run_until_complete(process_chunks_and_aggregate(chunked_files_directory))
    execution_time = time.time() - start_time
    print(f"Execution time: {execution_time} seconds")
    print("\nAggregated JSON:")
    print(json.dumps(aggregated_json, indent=2))

In [ ]:
import json

def format_transcript(input_file="finalized_transcript.json", output_file="formatted_transcript.txt"):
    # Read the JSON data from file
    with open(input_file, 'r') as file:
        aggregated_json = json.load(file)

    transcript_lines = []
    previous_entry = None  # To store the previous entry for comparison

    for entry in aggregated_json:
        if previous_entry and entry['id'] == previous_entry['id'] and entry['text'] == previous_entry['text']:
            continue  # Skip this entry if it's a duplicate of the previous one

        current_speaker = entry['speaker']
        formatted_time = entry['ts']  # Timestamp is already in HH:MM:SS format

        # Check for speaker changes to add additional spacing
        if previous_entry and current_speaker != previous_entry['speaker']:
            transcript_lines.append("\n\n")  # Add two newlines for speaker change
        elif transcript_lines:
            transcript_lines.append("\n")  # Add one newline for continued dialogue

        # Format the line as "(Timestamp) Speaker: Text"
        formatted_line = f"({formatted_time}) {current_speaker}: {entry['text']}"
        transcript_lines.append(formatted_line)

        previous_entry = entry  # Update the previous entry

    # Write the formatted lines to the output file
    with open(output_file, 'w') as file:
        file.writelines(transcript_lines)

    print(f"Formatted transcript saved to {output_file}")

# Call the function to start the formatting process
format_transcript()


Formatted transcript saved to formatted_transcript.txt
